In [13]:
from time import sleep
from pynq import Overlay
base = Overlay("/home/xilinx/pynq/overlays/prueba2sensor/pmodioconFIFO/sensor.bit")

In [15]:

MISO = "P18"
MOSI = "R17"
SCLK = "T16"
SS = "V18"

base.ps_gpio[MISO].setdirection("in")
#overlay.ps_gpio[MOSI].setdirection("out")
#overlay.ps_gpio[SCLK].setdirection("out")
#overlay.ps_gpio[SS].setdirection("in")

"""
cmd = [0xAA, 0x00, 0x00]

for value in cmd:
    base.ps_gpio["V18"].write(cmd)
"""


#overlay.ps_gpio[pin_number].write(1)
#ss_io = base.spi_rtl_ss_io[0] #V18
#sck_io = base.spi_rtl_sck_io #T16
#io1 = base.spi_rtl_io1_io #mosi R17
#io0 = base.spi_rtl_io0_io #miso P18

AttributeError: Could not find IP or hierarchy ps_gpio in overlay

In [16]:
base?

In [6]:
spi = base.axi_quad_spi_0


In [25]:
cmd = [0xAA, 0x00, 0x00]
spi.write(0x40,0x0a)
spi.write(0x28,0x04)
spi.write(0x1c,0)
spi.write(0x70,0xFFFFFFFF)

In [21]:
methods = [method for method in dir(spi) if callable(getattr(spi, method))]
print(methods)

['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_call', '_setup_packet_prototype', '_start_ert', '_start_sw', 'read', 'write']


In [23]:
spi.write?


In [14]:
#!/usr/bin/python
# Xilinx AXI quad SPI 

#from uio import Uio
from pynq import MMIO
# from ctypes import c_uint32
#import time

# SPI_regSpace = ol.ip_dict['axi_quad_spi_2']
# IP_BASE_ADDRESS = SPI_regSpace['phys_addr']
# IP_ADDRESS_RNGE = SPI_regSpace['addr_range']
# AxiQspi = MMIO(IP_BASE_ADDRESS, IP_ADDRESS_RNGE)

XSP_DGIER_OFFSET = 0x1C
XSP_IISR_OFFSET = 0x20
XSP_IIER_OFFSET = 0x28
XSP_SRR_OFFSET = 0x40
XSP_CR_OFFSET = 0x60
XSP_SR_OFFSET = 0x64
XSP_DTR_OFFSET = 0x68
XSP_DRR_OFFSET = 0x6C
XSP_SSR_OFFSET = 0x70
XSP_TFO_OFFSET = 0x74
XSP_RFO_OFFSET = 0x78
XSP_REGISTERS = [0x40, 0x60, 0x64, 0x68, 0x6c, 0x70, 0x74, 0x78, 0x1c, 0x20, 0x28]

XSP_SRR_RESET_MASK = 0x0A
XSP_SR_TX_EMPTY_MASK = 0x00000004
XSP_SR_TX_FULL_MASK	= 0x00000008
XSP_CR_TRANS_INHIBIT_MASK = 0x00000100
XSP_CR_LOOPBACK_MASK	= 0x00000001
XSP_CR_ENABLE_MASK	= 0x00000002
XSP_CR_MASTER_MODE_MASK	= 0x00000004
XSP_CR_CLK_POLARITY_MASK = 0x00000008
XSP_CR_CLK_PHASE_MASK	= 0x00000010
XSP_CR_TXFIFO_RESET_MASK = 0x00000020
XSP_CR_RXFIFO_RESET_MASK = 0x00000040
XSP_CR_MANUAL_SS_MASK	= 0x00000080
XSP_CR_TRANS_INHIBIT_MASK = 0x00000100

SLAVE_NO_SELECTION = 0xFFFFFFFF

def cnfg(AxiQspi, clk_phase=0, clk_pol=0):
	print("Configure device")
	AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
	AxiQspi.write(XSP_DGIER_OFFSET, 0)
	AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
	ControlReg = AxiQspi.read(XSP_CR_OFFSET)
	ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK
	AxiQspi.write(XSP_CR_OFFSET, ControlReg)
	ControlReg = AxiQspi.read(XSP_CR_OFFSET)
	if clk_phase == 1:
		ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
	else:
		ControlReg = ControlReg & ~XSP_CR_CLK_PHASE_MASK
	if clk_pol == 1:
		ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
	else:
		ControlReg = ControlReg & ~XSP_CR_CLK_POLARITY_MASK
	AxiQspi.write(XSP_CR_OFFSET, ControlReg)

	return 0

def xfer(packet, AxiQspi):
	resp = list()
	print("TransferData")
	for data in packet:
		AxiQspi.write(XSP_DTR_OFFSET, data)
		AxiQspi.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
		ControlReg = AxiQspi.read(XSP_CR_OFFSET)
		ControlReg = ControlReg & ~XSP_CR_TRANS_INHIBIT_MASK
		AxiQspi.write(XSP_CR_OFFSET, ControlReg)

		StatusReg = AxiQspi.read(XSP_SR_OFFSET)
		while (StatusReg & XSP_SR_TX_EMPTY_MASK) == 0:
			print("imbecil")
			StatusReg = AxiQspi.read(XSP_SR_OFFSET)

		print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(AxiQspi.read(XSP_RFO_OFFSET)))
		ControlReg = AxiQspi.read(XSP_CR_OFFSET)
		ControlReg = ControlReg | XSP_CR_TRANS_INHIBIT_MASK
		AxiQspi.write(XSP_CR_OFFSET, ControlReg)

	AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
	sleep(1)
	print("ReadResponse")
	RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
	while RxFifoStatus == 0:
		print("imbecil")
		#temp = AxiQspi.read(XSP_RFO_OFFSET)
		#print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(temp))
		temp = AxiQspi.read(XSP_DRR_OFFSET)
		print('XSP_DRR_OFFSET  : 0x{0:08x}'.format(temp))    
		resp.append(temp)
		RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
	return resp

In [16]:
cmd = [0xAA, 0x00, 0x00]
data = [0xFA, 0x00, 0x00,0x00,0x00,0x00,0X00]
press_counts = 0
temp_counts = 0
pressure = 0
temperature = 0
outputmax = 15099494
outputmin = 1677722
pmax = 1
pmin = 0


cnfg(spi)
while(1):
    cmdans = xfer(cmd, spi)
    dataans = xfer(data, spi)

    press_counts = dataans[3] + dataans[2] * 256 + dataans[1] * 65536
    temp_counts = dataans[6] + dataans[5] * 256 + dataans[4] * 65536
    temperature = (temp_counts * 200 / 16777215) - 50
    percentage = (press_counts / 16777215) * 100
    pressure = ((press_counts - outputmin) * (pmax - pmin)) / (outputmax - outputmin) + pmin

    # Imprimir los resultados
    print("Presión: {:.2f} bar".format(pressure))
    print("Temperatura: {:.2f} °C".format(temperature))
        

Configure device
TransferData
XSP_RFO_OFFSET  : 0x00000000
XSP_RFO_OFFSET  : 0x00000001
XSP_RFO_OFFSET  : 0x00000002
ReadResponse
imbecil
XSP_DRR_OFFSET  : 0x00000040
imbecil
XSP_DRR_OFFSET  : 0x00000080
imbecil
XSP_DRR_OFFSET  : 0x0000007e
TransferData
XSP_RFO_OFFSET  : 0x00000000
XSP_RFO_OFFSET  : 0x00000001
XSP_RFO_OFFSET  : 0x00000002
XSP_RFO_OFFSET  : 0x00000003
XSP_RFO_OFFSET  : 0x00000004
XSP_RFO_OFFSET  : 0x00000005
XSP_RFO_OFFSET  : 0x00000006
ReadResponse
imbecil
XSP_DRR_OFFSET  : 0x00000040
imbecil
XSP_DRR_OFFSET  : 0x00000080
imbecil
XSP_DRR_OFFSET  : 0x00000080
imbecil
XSP_DRR_OFFSET  : 0x00000012
imbecil
XSP_DRR_OFFSET  : 0x0000005a
imbecil
XSP_DRR_OFFSET  : 0x0000009d
imbecil
XSP_DRR_OFFSET  : 0x000000c2
Presión: 0.50 bar
Temperatura: 20.79 °C
TransferData
XSP_RFO_OFFSET  : 0x00000000
XSP_RFO_OFFSET  : 0x00000001
XSP_RFO_OFFSET  : 0x00000002
ReadResponse
imbecil
XSP_DRR_OFFSET  : 0x00000040
imbecil
XSP_DRR_OFFSET  : 0x00000080
imbecil
XSP_DRR_OFFSET  : 0x00000080
Transfe

KeyboardInterrupt: 

In [ ]:
from pynq import MMIO
from pynq import Overlay

# Cargar la overlay y encontrar la dirección base de la IP SPI
overlay = Overlay("/home/xilinx/pynq/overlays/prueba2sensor/sensor.bit")  # Reemplaza con la ruta a tu overlay.bit
axi_quad_spi = overlay.axi_quad_spi_0

XSP_DTR_OFFSET = 0x68
XSP_SSR_OFFSET = 0x70
XSP_SR_OFFSET = 0x64
XSP_CR_OFFSET = 0x60

SLAVE_NO_SELECTION = 0xFFFFFFFF
XSP_SR_TX_EMPTY_MASK = 0x00000004

def configure_spi(axi_quad_spi, clk_phase=0, clk_pol=0):
    print("Configurando el dispositivo SPI")
    axi_quad_spi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    
    # Habilitar el modo maestro, configurar fase y polaridad del reloj, habilitar manualmente el esclavo (SS), y habilitar la IP SPI
    control_reg = axi_quad_spi.read(XSP_CR_OFFSET)
    control_reg |= (XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK)
    
    if clk_phase == 1:
        control_reg |= XSP_CR_CLK_PHASE_MASK
    else:
        control_reg &= ~XSP_CR_CLK_PHASE_MASK
    
    if clk_pol == 1:
        control_reg |= XSP_CR_CLK_POLARITY_MASK
    else:
        control_reg &= ~XSP_CR_CLK_POLARITY_MASK
    
    axi_quad_spi.write(XSP_CR_OFFSET, control_reg)

def transfer_data(packet, axi_quad_spi):
    print("Transfiriendo datos")
    
    for data in packet:
        # Escribir el dato en el registro de transmisión
        axi_quad_spi.write(XSP_DTR_OFFSET, data)
        
        # Seleccionar el esclavo
        axi_quad_spi.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
        print('hola')
        # Habilitar la transferencia de datos
        control_reg = axi_quad_spi.read(XSP_CR_OFFSET)
        control_reg &= ~XSP_CR_TRANS_INHIBIT_MASK
        axi_quad_spi.write(XSP_CR_OFFSET, control_reg)
        
        resp =list()
        # Esperar hasta que el registro de estado indique que la transmisión está completa
        status_reg = axi_quad_spi.read(XSP_SR_OFFSET)&0x01
        while (status_reg & XSP_SR_TX_EMPTY_MASK) == 0:
            status_reg = axi_quad_spi.read(XSP_SR_OFFSET)
            data2 = AxiQspi.read(XSP_DRR_OFFSET)
            resp.append(data2)
            status_reg = axi_quad_spi.read(XSP_SR_OFFSET)&0x01
            print(data2)
            print("imbec")

    # Deseleccionar el esclavo
    axi_quad_spi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    return resp



In [ ]:

    configure_spi(axi_quad_spi, clk_phase=0, clk_pol=0)
    press_counts = 0
    temp_counts = 0
    pressure = 0
    temperature = 0
    outputmax = 15099494
    outputmin = 1677722
    pmax = 1
    pmin = 0
    percentage = 0
    while True:
        # Lista de comandos de lectura (debes adaptarlos a tu sensor específico)
        cmd = [0xAA, 0x00, 0x00]
        
        # Realizar la transferencia de comandos
        transfer_data(cmd, axi_quad_spi)
        
        # Esperar el tiempo necesario para la conversión
        # (debes ajustarlo según las especificaciones del sensor)
        sleep(10)  # Ajusta el tiempo de espera adecuado
        
        # Lista para almacenar la respuesta del sensor
        data = [0, 0, 0, 0, 0, 0, 0]
        
        # Realizar la transferencia de datos
        print(data)
        transfer_data(data, axi_quad_spi)
        print(data)
        # Procesar los datos y calcular la presión y la temperatura
        press_counts = data[3] + data[2] * 256 + data[1] * 65536
        temp_counts = data[6] + data[5] * 256 + data[4] * 65536
        temperature = (temp_counts * 200 / 16777215) - 50
        percentage = (press_counts / 16777215) * 100
        pressure = ((press_counts - outputmin) * (pmax - pmin)) / (outputmax - outputmin) + pmin
        
        # Imprimir los resultados
        print("Presión: {:.2f} bar".format(pressure))
        print("Temperatura: {:.2f} °C".format(temperature))
        
        # Ajusta el tiempo de espera adecuado antes de realizar otra lectura
        sleep(1000)  # Espera 1 segundo antes de la próxima lectura

Configurando el dispositivo SPI
Transfiriendo datos
hola
hola
hola
[0, 0, 0, 0, 0, 0, 0]
Transfiriendo datos
hola
hola
hola
hola
hola
hola
hola
[0, 0, 0, 0, 0, 0, 0]
Presión: -0.13 bar
Temperatura: -50.00 °C
